In [1]:
import sys

sys.path.append("../")

from loaders.LHCO import LhcoRnDLoader
from analysis.scalar import HLFAnalysis
from models.nn import PaeBuilder
from utils import load_json, dump_json

import tensorflow as tf
import numpy as np

In [2]:
gpus = tf.config.list_physical_devices('GPU')
if gpus:
  try:
    # Currently, memory growth needs to be the same across GPUs
    for gpu in gpus:
      tf.config.experimental.set_memory_growth(gpu, True)
    logical_gpus = tf.config.experimental.list_logical_devices('GPU')
    print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
  except RuntimeError as e:
    # Memory growth must be set before GPUs have been initialized
    print(e)

1 Physical GPUs, 1 Logical GPUs


In [3]:
import glob

configs_folder = '../configs/analysis'
config_files = glob.glob(configs_folder+'/*.json')
config_files

['../configs/analysis/trial.json']

In [4]:
config_files[0].replace('.', '/').split('/')[-2]

'trial'

In [5]:
import os

plots_dir = './figures/'
if not os.path.exists(plots_dir):
    os.mkdir(plots_dir)
results_history = {
 'id': [],
 'config': [],
 'js_div_train': [],
 'js_div_test': [],
 'sig_eff': [],
 'bkg_rej': [],
 'auc': []}

In [8]:
 for file in config_files:
    analysis_cfg = load_json(file)
    analysis_cfg

    config = analysis_cfg.copy()

    loader_json = analysis_cfg.pop('ANA:loader')
    dataset_json = analysis_cfg.pop('ANA:dataset')
    density_estimator = analysis_cfg.pop('ANA:estimator')
    prc = analysis_cfg.pop('ANA:percentile')

    loader = LhcoRnDLoader.from_json(loader_json)
    dataset_cfg = load_json(dataset_json)
    dataset = loader.make_full_dataset(**dataset_cfg)

    builder = PaeBuilder()

    pae, ae_train, nf_train = builder.from_json(analysis_cfg)
    task = HLFAnalysis(pae, dataset=dataset)
    task.reweighting(estimator=density_estimator, fit_key='mjj_train')
    if 'cond' in config['nf_model']:
        task.make_cond_inputs(['mjj_train', 'mjj_test', 'mjj_valid'])
    task.train(ae_train,nf_train)
    result = task.evaluate(prc = prc)
    for key in result.keys():
        results_history[key].append(result[key])
    
    id = config_files[0].replace('.', '/').split('/')[-2]
    results_history['id'].append(id)
    results_history['config'].append(config)
    task.plot_training(plots_dir+id+'_train.png')
    task.plot_latent_space(plots_dir+id+'_latent.png')



[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'), PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
Epoch 1/60
800/800 [==============================] - 6s 6ms/step - loss: 627.8270 - val_loss: 308863.5625
Epoch 2/60
800/800 [==============================] - 1s 1ms/step - loss: 472.6789 - val_loss: 306993.5000
Epoch 3/60
800/800 [==============================] - 1s 1ms/step - loss: 439.6953 - val_loss: 279228.9688
Epoch 4/60
800/800 [==============================] - 1s 1ms/step - loss: 446.1837 - val_loss: 241646.3594
Epoch 5/60
800/800 [==============================] - 1s 2ms/step - loss: 438.9169 - val_loss: 239815.4219
Epoch 6/60
800/800 [==============================] - 1s 2ms/step - loss: 385.4571 - val_loss: 219290.9844
Epoch 7/60
800/800 [==============================] - 1s 1ms/step - loss: 379.0493 - val_loss: 219682.4375
Epoch 8/60
800/800 [==============================] - 1s 2ms/step - loss: 367.0194 - val_loss: 218906.7500
Epoch 9/60


In [9]:
results_history

{'id': ['trial'],
 'config': [{'ANA:loader': '../configs/loader/default_rnd.json',
   'ANA:dataset': '../configs/loader/default_dataset.json',
   'ANA:estimator': 'gmm',
   'ANA:percentile': 95,
   'ae_model': 'dense_ae',
   'nf_model': 'cond_maf',
   'AE:input_dim': 47,
   'AE:encoding_dim': 10,
   'AE:units': [30, 20, 15],
   'AE:hidden_activation': 'relu',
   'AE:output_activation': 'sigmoid',
   'AE:weight_reg': {'l1': 1e-06},
   'AE:bias_reg': {},
   'AE:output_reg': {},
   'NF:n_dims': 10,
   'NF:n_layers': 5,
   'NF:units': [32, 32, 32, 32],
   'ae_optimizer': 'adam',
   'nf_optimizer': 'adam',
   'ae_optimizer_kwargs': {'lr': 0.001},
   'nf_optimizer_kwargs': {'lr': 0.005},
   'ae_callbacks': ['reduce_lr_on_plateau'],
   'ae_callbacks_kwargs': [{'factor': 0.2, 'patience': 10, 'verbose': 1}],
   'nf_callbacks': ['reduce_lr_on_plateau'],
   'nf_callbacks_kwargs': [{'factor': 0.2, 'patience': 5, 'verbose': 1}],
   'ae_epochs': 60,
   'ae_batch_size': 200,
   'nf_epochs': 50,
   'n